In [16]:
import mediapipe as mp
import numpy as np
import pandas as pd
import cv2
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2

In [44]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = solutions.pose


# Selected values of pose landmarks corresponding to MediaPipe library
values = [
    0, # NOSE
    11, # LEFT_SHOULDER
    12, # RIGHT_SHOULDER
    13, # LEFT_ELBOW
    14, # RIGHT_ELBOW
    15, # LEFT_WRIST
    16, # RIGHT_WRIST
    19, # LEFT_INDEX
    20, # RIGHT_INDEX
    23, # LEFT_HIP
    24, # RIGHT_HIP
    25, # LEFT_KNEE
    26, # RIGHT_KNEE
    27, # LEFT_ANKLE
    28, # RIGHT_ANKLE
    31, # LEFT_FOOT_INDEX
    32, # RIGHT_FOOT_INDEX
]


# Create a dictionary to store landmark names and new values
landmarks = dict()
for index, value in enumerate(values):
    # Extract pose landmark names
    name = mp_pose.PoseLandmark(value).name
    # Complete the dictionary with names and values
    landmarks[value] = {
        'landmark_name' : name,
        'new_value' : index
    }

# # Add a completely new pose landmark
# landmarks[max(landmarks) + 1] = {
#     'landmark_name' : 'THORAX',
#     'new_value' : len(landmarks)
# }


# Create a custom connections set
connections = set()
for connection in mp_pose.POSE_CONNECTIONS:
    # Extract old values from POSE_CONNECTIONS
    v1, v2 = connection
    # Check if the values are expected
    if v1 in values and v2 in values:
        # Create connections for new values
        connections.add(
            (landmarks[v1]['new_value'], landmarks[v2]['new_value'])
        )

# # Add a new connection
# connections.add(
#     (landmarks[0]['new_value'], landmarks[33]['new_value'])
#         )

In [18]:
import numpy as np
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2


mp_pose = solutions.pose

def landmark2array(landmark):
    return np.array(
        [
            landmark.x,
            landmark.y,
            landmark.z,
            landmark.visibility
        ]
    )

def array2landmark(array):
    return landmark_pb2.NormalizedLandmark(
        x=array[0],
        y=array[1],
        z=array[2],
        visibility=array[3]
    )

def get_custom_landmarks(indexes, landmarks):
    """
    
    """
    # Create customize landmarks list
    custom_landmarks = landmark_pb2.NormalizedLandmarkList()

    # Extend list by chosen landmarks
    custom_landmarks.landmark.extend(
        [landmarks.landmark[index] for index in indexes])

    # Calculate the coordinates of neck landmark
    left_shoulder = landmark2array(
        landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER])
    right_shoulder = landmark2array(
        landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER])
    neck = np.mean([left_shoulder, right_shoulder], axis=0)
    neck_landmark = array2landmark(neck)

    # Add neck landmark to custom list
    custom_landmarks.landmark.add().CopyFrom(neck_landmark)

    return custom_landmarks

In [33]:
landmark_names = [value['landmark_name'] for value in landmarks.values()]
# [name.lower() + '_' + char for name in landmark_names for char in ('x', 'y', 'z')]

['nose_x',
 'nose_y',
 'nose_z',
 'left_shoulder_x',
 'left_shoulder_y',
 'left_shoulder_z',
 'right_shoulder_x',
 'right_shoulder_y',
 'right_shoulder_z',
 'left_elbow_x',
 'left_elbow_y',
 'left_elbow_z',
 'right_elbow_x',
 'right_elbow_y',
 'right_elbow_z',
 'left_wrist_x',
 'left_wrist_y',
 'left_wrist_z',
 'right_wrist_x',
 'right_wrist_y',
 'right_wrist_z',
 'left_index_x',
 'left_index_y',
 'left_index_z',
 'right_index_x',
 'right_index_y',
 'right_index_z',
 'left_hip_x',
 'left_hip_y',
 'left_hip_z',
 'right_hip_x',
 'right_hip_y',
 'right_hip_z',
 'left_knee_x',
 'left_knee_y',
 'left_knee_z',
 'right_knee_x',
 'right_knee_y',
 'right_knee_z',
 'left_ankle_x',
 'left_ankle_y',
 'left_ankle_z',
 'right_ankle_x',
 'right_ankle_y',
 'right_ankle_z',
 'left_foot_index_x',
 'left_foot_index_y',
 'left_foot_index_z',
 'right_foot_index_x',
 'right_foot_index_y',
 'right_foot_index_z',
 'thorax_x',
 'thorax_y',
 'thorax_z']

In [38]:
chosen_indexes = [key for key in landmarks.keys()]

In [45]:
get_custom_landmarks(chosen_indexes, landmarks)

AttributeError: 'dict' object has no attribute 'landmark'

In [46]:
# source posibilities: path, int
# source = r'C:/Users/nemet/Desktop/XPC_2023/MVI_7011.mp4'
source = r'C:/Users/nemet/Desktop/XPC_2023/test8.mp4'
# source = 0

# background bool: Flase, True
background = True

# output posibilities: 'landmarks', 'mask', None
output = 'landmarks'

In [47]:
column_names = ['timestamp'] + [name.lower() + '_' + char for name in landmark_names for char in ('x', 'y', 'z')]

data = pd.DataFrame(columns = column_names, dtype = float)

time = 0
cap = cv2.VideoCapture(source)

# Setup MediaPipe instance
with mp_pose.Pose(
    min_detection_confidence=0.75,
    min_tracking_confidence=0.75,
    enable_segmentation=True
) as pose:
    while cap.isOpened():
        ret, image = cap.read()
        image_shape = image.shape


        # Camera condition -> if selfie or web camera is chosen then flip image
        if source == 0:
            image = cv2.flip(image, 1)
    
        # Recolor image for image processing
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Image MediaPipe processing -> detection
        results = pose.process(image)

        # Recolor back to BGR for visualization
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # create a fake background
        fake_background = np.zeros(shape=image_shape, dtype='uint8')
        # use a fake background if necessary
        if not background:
            image = fake_background

        # Image processing for different outputs
        if output == 'landmarks':
            default_landmarks = results.pose_landmarks

            if default_landmarks:
                time += 1

                # Get custom landmarks and create new connection
                custom_landmarks = get_custom_landmarks(chosen_indexes, default_landmarks)
                #custom_connections.add((0, len(custom_landmarks.landmark) - 1))

                # Prepare a single record storage
                record = np.array([time])

                for landmark in custom_landmarks.landmark:
                    # Extract pose landmarks coordinates and store as array
                    coordinates = landmark2array(landmark)[:3]
                    record = np.concatenate(
                        [
                            record,
                            coordinates
                        ]
                    )

                # Save pose landmarks coordinates in time as DataFrame
                data = pd.concat([data, pd.DataFrame([record], columns = column_names)], ignore_index=True)

                # Draw customize landmarks on image
                mp_drawing.draw_landmarks(
                    image,
                    landmark_list=custom_landmarks,
                    connections=connections,
                )


        elif output == 'mask':
            image = results.segmentation_mask
            if image is None:
                image = fake_background


        cv2.imshow(f'background: {background}, output: {output}', image)

        if cv2.waitKey(10) & 0xFF == ord("q"):
            break

        if not cap.isOpened():
            exit()
            
cap.release()
cv2.destroyAllWindows()

IndexError: list index out of range